In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import requests
import datetime
from urllib.parse import unquote

In [10]:
# API 내 오늘 날짜 -1일 까지만 조회가 가능
today = pd.datetime.now()
today_before = today-pd.Timedelta(1, unit='d')
# 최초 정보 수집일 설정
st_day_1st = pd.datetime(2000, 1,1)
# 최초 정보수집일
st_day = st_day_1st
# API 호출 시 페이지별로 100개의 데이터를 추출하여, 몇 페이지로 구성되는지 계산 진행
count_day = (today_before - st_day_1st)/100

# 데이터 저장을 위한 DataFrame 생성 (내부에서 필요한 데이터만 추출)
# ※ 실제 리턴값은 API 문서 확인
item = ['tm', 'avgTa', 'sumRnDur', 'sumRn', 'maxInsWs', 'avgWs', 'minRhmHrmt', 'avgPa', 'ssDur', 'sumSsHr', 
'sumGsr', 'ddMefs', 'avgTca', 'avgLmac', 'iscs', 'sumFogDur']
wth_df = pd.DataFrame(columns = item)

# API 호출을 위해 datetime형 데이터 yyyymmdd 만 추출하여 문자열로 변경
st_day_str = str(st_day)[:10].replace('-','')
today_before_str = str(today_before)[:10].replace('-','')

### API호출을 위해 받은 인증키값 인코딩
key = 'enj%2B1kAlWLkkD8GcOtQTEdbOHBQrK2j%2BmnT4a4ykntUGNQh3Wq0STrtWqnN%2FcpUGEx17ww8KFm%2FaIEufvy5O6Q%3D%3D'
key_encode = unquote(key)
try : 
# 날씨 데이터 가져오기
    for i in range(1, count_day.days + 2) : 
        ### API 호출 (페이지를 옮겨가며 데이터 저장)
        url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'
        params = {'serviceKey':key_encode, 'pageNo':i, 'numOfRows':'100', 'dataType':'JSON', \
                 'dataCd':'ASOS', 'dateCd':'DAY', 'startDt':st_day_str, 'endDt':today_before_str, \
                 'stnIds':'108'}

        ### 호출을 통해 받은 데이터 pandas 로 변환
        response = requests.get(url, params=params)
        ### 읽은 json 데이터 df형태로 변경
        print(i)
        df = pd.read_json(response.text)
        ### json 데이터 내 원하는 항목이 있는 값 추출
        data_len = df['response']['body']['items']['item']

        # 페이지별로 추출한 데이터의 길이만큼 for문 작성
        # (기본적으로 페이지당 100개의 데이터이나 마지막 페이지의 경우 100개가 아닐 수 있어 데이터 길이로 작성)
        for j in range(len(data_len)) :
            # 추출한 데이터 값 내 필요한 항목만 따로 데이터 추출
            data = [data_len[j]['tm'], # 날짜
                    data_len[j]['avgTa'], # 평균기온
                    data_len[j]['sumRnDur'], # 강수계속시간
                    data_len[j]['sumRn'], # 일 강수량
                    data_len[j]['maxInsWs'], # 최대 순간 풍속
                    data_len[j]['avgWs'], # 평균풍속
                    data_len[j]['minRhmHrmt'], # 평균 상대 습도
                    data_len[j]['avgPa'], # 평균현지기압
                    data_len[j]['ssDur'], # 가조시간
                    data_len[j]['sumSsHr'], # 합계 일조 시간
                    data_len[j]['sumGsr'], # 합계 일사량
                    data_len[j]['ddMefs'], # 일 최심신적설
                    data_len[j]['avgTca'], # 평균 전운량
                    data_len[j]['avgLmac'], # 평균 중하층운량
                    data_len[j]['iscs'], # 일기 현상
                    data_len[j]['sumFogDur']] # 안개 계속시간
            # 추출한 데이터 DataFrame에 저장
            wth_df = wth_df.append(pd.Series(data, index=wth_df.columns), ignore_index=True)
except : 
    pass
# 저장한 데이터 마지막 데이터 수집일 기준으로 라벨링하여 경로 내 저장
today_str = str(today)[:10].replace('-','')
path = './wth_data_{}.csv'.format(today_str)
# 저장 시 한글깨짐 방지를 위해 인코딩 진행
wth_df.to_csv(path,encoding="utf-8-sig")

<ipython-input-10-05e0e9ecface>:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  today = pd.datetime.now()
<ipython-input-10-05e0e9ecface>:5: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  st_day_1st = pd.datetime(2000, 1,1)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
